### Data Extraction

Preprocess and extract features from a corpus of NIPS and ICLR papers.
For each paper, we fetch the title, decision, conference, abstract, introduction, and reviews.


In [9]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# Imports
import sys
import os
import time
sys.path.insert(0, "/content/drive/MyDrive/CS 282 Project/Checkpoint 2/cs282-project/data_extraction")
from utils import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erichansen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/erichansen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Global Variables
# prefix = "/content/drive/MyDrive/CS 282 Project/Checkpoint 2/cs282-project/data_extraction/"
prefix = "./"
keywords_path = prefix + "keywords.txt"
parameters_path = prefix + "parameters.txt"
# workdir = '/content/drive/MyDrive/CS 282 Project/Checkpoint 2/'
workdir = "./"

In [19]:
def get_full_text_v2(conference, year, file_num):
    kind = 'content'
    fname = workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + kind+'/' + conference + '_' + year + '_' + file_num + '_' + kind + '.json'
    
    return get_full_text(fname)

def get_full_text(paper_json):
    full_text = ""
    with open(paper_json, 'r', encoding='utf8') as f:
        content_dict = json.loads(f.read())
        sections = content_dict.get('metadata').get('sections')
        for section in sections:
            heading: str = section.get('heading')
            text: str = section.get('text')
            if heading is not None:
                if heading.upper().__contains__('ACKNOW') or heading.upper().__contains__('APPEN'):
                    break
            if text is not None and len(text) > 0:
                full_text += text + " "
    full_text = full_text.replace("\n", " ").encode("utf-8", "ignore").decode("utf-8").strip()
    return full_text

In [20]:
def get_abstract(paper_json):
    """Extract the abstract as a string from a paper's json object
        Return None if no valid introduction found."""
    with open(paper_json, 'r', encoding='utf8') as f:
        content_dict = json.loads(f.read())
    abstract = content_dict.get('metadata').get('abstractText')
    if not abstract:
        return None
    abstract = abstract.replace("\n", " ").encode("utf-8", "ignore").decode("utf-8").strip()
    return abstract

def get_intro(paper_json):
    """Extract the introduction as a string from a paper's json object.
        Return None if no valid introduction found."""
    with open(paper_json, 'r', encoding='utf8') as f:
        content_dict = json.loads(f.read())
    intro_section = content_dict.get('metadata').get('sections')[0]
    if not intro_section["heading"] or not intro_section["text"] or not "introduction" in intro_section["heading"].lower():
        return None
    intro = intro_section["text"]
    intro = intro.replace("\n", " ").encode("utf-8", "ignore").decode("utf-8").strip()
    return intro

def extract_data(conference, year, file_num):
    """Helper fundtion to extract features from a specific paper"""
    t0 = time.time()
    paper_dict = {}

    # Check whether content and review info exist, if not, exclude paper
    kind = 'paper'
    fname = workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + kind+'/' + conference + '_' + year + '_' + file_num + '_' + kind + '.json'
    with open(fname, 'r', encoding='utf8') as f:
        content_dict = json.loads(f.read())
    if content_dict['hasContent'] != 'true' or content_dict['hasReview'] != 'true':
        return None

    # Get basic paper information
    paper_dict['title'] = content_dict['title']
    paper_dict['decision'] = content_dict['decision']
    paper_dict['conference'] = content_dict['conference']

    # Get longform paper text and use the extractor to condense
    kind = 'content'
    fname = workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + kind+'/' + conference + '_' + year + '_' + file_num + '_' + kind + '.json'

    # Get abstract for paper. Drop paper if none exists.
    paper_dict["abstract"] = get_abstract(fname)
    if not paper_dict["abstract"]:
        return None

    # Get introduction for paper. Drop paper if none exists.
    paper_dict["intro"] = get_intro(fname)
    if not paper_dict["intro"]:
        return None

    # Get reviews.
    kind = 'review'
    fname = workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + kind+'/' + conference + '_' + year + '_' + file_num + '_'+ kind + '.json'
    with open(fname, 'r', encoding='utf8') as f:
        content_dict = json.loads(f.read())
    reviews = []
    for r in content_dict['reviews']:
        reviews.append(r['review'])
    paper_dict['reviews'] = reviews
    
    # Get Full Text
    paper_dict['full_text'] = get_full_text_v2(conference, year, file_num)

    # Print processing time for currenet paper.
    t1 = time.time()
    print(fname)
    print('finished in ' + str(t1 - t0) + ' seconds')

    # Add to list to be converted to JSON
    return paper_dict


In [21]:
## reviews
## arxiv.cs

In [22]:
peer_read_workdir = './'
conferences = {
    
}


In [23]:
workdir = './'

In [24]:
# Iterate through papers by conference, year, and number. Fetch each of
# their features and add to json_list.

conferences = {
    "ICLR": ["2017", "2018", "2019", "2020"],
    "NIPS": ["2016", "2017", "2018", "2019"]
}

json_list = []
for conference in conferences:
    for year in conferences[conference]:
        kind = "content"
        fname = workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + kind+'/'
        files = os.listdir(fname)
        file_nums = [x.split('_')[-2] for x in files]
        for file_num in  file_nums:
            extraction = extract_data(conference, year, file_num)
            if extraction:
                json_list.append(extraction)

./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_9_review.json
finished in 0.002753019332885742 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_266_review.json
finished in 0.0025680065155029297 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_233_review.json
finished in 0.0024068355560302734 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_299_review.json
finished in 0.002637147903442383 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_344_review.json
finished in 0.0026018619537353516 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_311_review.json
finished in 0.0028281211853027344 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_460_review.json
finished in 0.002788066864013672 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_261_review.json
finished in 0.002640962600708008 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_234_review.json
finished in 0.004606962203979492 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_343_review.json

./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_287_review.json
finished in 0.002438068389892578 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_110_review.json
finished in 0.0018532276153564453 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_142_review.json
finished in 0.0017168521881103516 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_117_review.json
finished in 0.0018749237060546875 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_18_review.json
finished in 0.002777099609375 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_387_review.json
finished in 0.002023935317993164 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_409_review.json
finished in 0.002496004104614258 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_380_review.json
finished in 0.0019168853759765625 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_208_review.json
finished in 0.0019297599792480469 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_135_review.json


./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_79_review.json
finished in 0.002507925033569336 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_86_review.json
finished in 0.002146005630493164 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_123_review.json
finished in 0.001966238021850586 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_176_review.json
finished in 0.0017042160034179688 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_189_review.json
finished in 0.0019130706787109375 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_418_review.json
finished in 0.0018520355224609375 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_396_review.json
finished in 0.0018088817596435547 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_219_review.json
finished in 0.001644134521484375 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_391_review.json
finished in 0.002093791961669922 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_136_review.json

./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_246_review.json
finished in 0.0019049644470214844 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_213_review.json
finished in 0.0018508434295654297 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_363_review.json
finished in 0.002001047134399414 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_447_review.json
finished in 0.001873016357421875 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_412_review.json
finished in 0.0024242401123046875 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_241_review.json
finished in 0.002129077911376953 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_214_review.json
finished in 0.002165079116821289 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_159_review.json
finished in 0.002371072769165039 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_56_review.json
finished in 0.0020890235900878906 seconds
./dataset/ICLR_2017/ICLR_2017_review/ICLR_2017_51_review.json

./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_511_review.json
finished in 0.003061056137084961 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_838_review.json
finished in 0.0020749568939208984 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_37_review.json
finished in 0.0019540786743164062 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_466_review.json
finished in 0.0018410682678222656 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_892_review.json
finished in 0.0021600723266601562 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_62_review.json
finished in 0.0021238327026367188 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_433_review.json
finished in 0.0021331310272216797 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_342_review.json
finished in 0.0026140213012695312 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_317_review.json
finished in 0.0030868053436279297 seconds
./dataset/ICLR_2018/ICLR_2018_review/ICLR_2018_749_review.

...

./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_746_review.json
finished in 0.002238750457763672 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1330_review.json
finished in 0.0019030570983886719 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1414_review.json
finished in 0.0035479068756103516 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_115_review.json
finished in 0.0020821094512939453 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_140_review.json
finished in 0.001993894577026367 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_631_review.json
finished in 0.0016210079193115234 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1212_review.json
finished in 0.001940011978149414 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_664_review.json
finished in 0.00203704833984375 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1362_review.json
finished in 0.001967191696166992 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_714_review

./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1423_review.json
finished in 0.003025054931640625 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1307_review.json
finished in 0.002015829086303711 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1352_review.json
finished in 0.0019140243530273438 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_724_review.json
finished in 0.002134084701538086 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1222_review.json
finished in 0.0021109580993652344 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_654_review.json
finished in 0.002485036849975586 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1277_review.json
finished in 0.002454996109008789 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_601_review.json
finished in 0.0018739700317382812 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1288_review.json
finished in 0.0021200180053710938 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_170_rev

./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_625_review.json
finished in 0.0020940303802490234 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1253_review.json
finished in 0.0021927356719970703 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_101_review.json
finished in 0.002023935317993164 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_755_review.json
finished in 0.0018627643585205078 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1323_review.json
finished in 0.0028181076049804688 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_700_review.json
finished in 0.002187967300415039 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1376_review.json
finished in 0.0019960403442382812 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_688_review.json
finished in 0.0018908977508544922 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1201_review.json
finished in 0.0022819042205810547 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_677_re

./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_471_review.json
finished in 0.0022859573364257812 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1052_review.json
finished in 0.0027947425842285156 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_424_review.json
finished in 0.0015568733215332031 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_69_review.json
finished in 0.002274036407470703 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_355_review.json
finished in 0.0019350051879882812 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_300_review.json
finished in 0.002132892608642578 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_270_review.json
finished in 0.001924753189086914 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_225_review.json
finished in 0.0016529560089111328 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1122_review.json
finished in 0.0021162033081054688 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_554_review

./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_797_review.json
finished in 0.002003908157348633 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_768_review.json
finished in 0.0024101734161376953 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1269_review.json
finished in 0.001971006393432617 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1296_review.json
finished in 0.0020437240600585938 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_191_review.json
finished in 0.0021691322326660156 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_1319_review.json
finished in 0.0022008419036865234 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_790_review.json
finished in 0.0017616748809814453 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_618_review.json
finished in 0.002032041549682617 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_169_review.json
finished in 0.001981973648071289 seconds
./dataset/NIPS_2019/NIPS_2019_review/NIPS_2019_196_revie

In [27]:
all_with_full_text_df = pd.DataFrame(json_list)

In [29]:
all_with_full_text_df.to_csv(workdir + 'all_with_full_text.csv', index=False)

In [30]:
all_with_full_text_df.to_csv('/Users/erichansen/Desktop/Classes/CS282/ReviewAdvisor/tagger/all_with_full_text.csv',
                             index=False)

In [20]:
# Writing to JSON file

# JSON format
# { 
#   "title": str
#   "decision": str
#   "conference": str
#   "abstract": str
#   "intro": str
#   "reviews": list of strs
# }

json_object = json.dumps(json_list, indent=4)
with open("/content/drive/MyDrive/CS 282 Project/Checkpoint 2/abstract_and_intro_data.json", "w") as outfile:
    outfile.write(json_object)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CS 282 Project/Checkpoint 2/abstract_and_intro_data.json'

# Pull Confidence, Ratings, Data, etc.

In [1]:
import pandas as pd 
import numpy as np

In [2]:
workdir = './'

NameError: name 'workdir' is not defined

In [ ]:
df = pd.read_csv(wo)

In [4]:
def generate_uid(jsn):
    uid = f"{jsn['conference']}_{jsn['year']}_{jsn['title']}"
    
    return uid

In [5]:
workdir = './'

In [9]:
def get_fname(conference, year, file_num, kind):
    return (workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + kind + '/' + 
            conference + '_' + year + '_' + file_num + '_' + kind + '.json'
           )

In [54]:
def extract_paper_metadata(conference, year, file_num):
    p_fname = get_fname(conference, year, file_num, 'paper')
    ct_fname = get_fname(conference, year, file_num, 'content') 
    
    entry = {'pid': '_'.join([conference, year, file_num])}
    
    url = ''
    authors = ''
    titlee = ''
    with open(p_fname, "r") as p_infile:
        p_data = json.load(p_infile)
        if 'url' in p_data:
            url = p_data['url']
        if 'authors' in p_data:
            authors = ', '.join(p_data['authors'])
        title = p_data['title']
        
    p_len = 0
    conclusion = ''
    source = ''
    emails = ''
    with open(ct_fname, "r") as c_infile:
        ct_data = json.load(c_infile)
        for section in ct_data['metadata']['sections']:
            p_len += len(section['text'].split())
#             print(section)
            if section['heading'] and 'conclusion' in section['heading'].lower():
                conclusion = section['text']
        if 'source' in ct_data['metadata']:
            source = ct_data['metadata']['source']
        if 'emails' in ct_data['metadata']:
            emails = ', '.join(ct_data['metadata']['emails'])
    
    entry.update({
        'paper_len': p_len,
        'conclusion': conclusion,
        'conclusion_len': len(conclusion),
        'url': url,
        'emails': emails,
        'source': source,
        'authors': authors,
        'title': title
    })
    
    return entry

In [55]:
def extract_reviews_metadata(conference, year, file_num):
    r_fname = get_fname(conference, year, file_num, 'review')
    entries = []
    
    confidence_str = ''
    rating_str = ''
    with open(r_fname, "r") as r_infile:
        r_data = json.load(r_infile)
        for i, review in enumerate(r_data['reviews']):
            entry = {}
            if 'confidence' in review:
                confidence_str = review['confidence']
            if 'rating' in review:
                rating_str = review['rating']

            entry.update({
                'pid': '_'.join([conference, year, file_num]),
                'rid': str(i+1),
                'uid': '_'.join([conference, year, file_num]) + '_r' + str(i+1),
                'confidence_str': confidence_str,
                'rating_str': rating_str,
                'review': review['review']
            })
            
            entries.append(entry)
    
    return entries

In [56]:
conferences = {
    "ICLR": ["2017", "2018", "2019", "2020"],
    "NIPS": ["2016", "2017", "2018", "2019"]
}
kind = 'review'

json_list = []
complete_entries = []
for conference in conferences:
    for year in conferences[conference]:
        kind = "content"
        review_dirname = workdir + 'dataset/' + conference + '_' + year + '/' + conference + '_' + year + '_' + 'review/'
        files = os.listdir(review_dirname)

        file_nums = [x.split('_')[-2] for x in files]
        
        for file_num in  file_nums:
            try:
                paper_metadata = extract_paper_metadata(conference, year, file_num)
                review_entries = extract_reviews_metadata(conference, year, file_num)
                for entry in review_entries:
                    assert(entry['pid'] == paper_metadata['pid'])
                    entry.update(paper_metadata)
                    complete_entries.append(entry)
            except Exception as e:
                print(str(e))

[Errno 2] No such file or directory: './dataset/ICLR_2017/ICLR_2017_content/ICLR_2017_318_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2017/ICLR_2017_content/ICLR_2017_240_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2017/ICLR_2017_paper/ICLR_2017_419_paper.json'
[Errno 2] No such file or directory: './dataset/ICLR_2018/ICLR_2018_content/ICLR_2018_852_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2018/ICLR_2018_content/ICLR_2018_678_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2018/ICLR_2018_content/ICLR_2018_487_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2019/ICLR_2019_content/ICLR_2019_638_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2019/ICLR_2019_content/ICLR_2019_1207_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2019/ICLR_2019_content/ICLR_2019_881_content.json'
[Errno 2] No such file or directory: './dataset/ICLR_2019/ICLR_2019

In [57]:
complete_df = pd.DataFrame(complete_entries)

In [58]:
merged_df = all_df.merge(complete_df, on=['title', 'review'], how='inner')

In [63]:
def get_confidence(c_str):
    if c_str == '':
        return np.nan
    splits = c_str.split('-')
    if len(splits) > 0:
        return splits[0]
    else:
        return np.nan    
    
def get_rating(c_str):
    if c_str == '':
        return np.nan
    splits = c_str.split(':')
    if len(splits) > 0:
        return splits[0]
    else:
        return np.nan    
    
def get_acceptance(a_str):
    if 'accept' in a_str.lower():
        return 1.0
    return 0.0

In [64]:
merged_df['confidence'] = merged_df['confidence_str'].apply(get_confidence)
merged_df['rating'] = merged_df['rating_str'].apply(get_rating)
merged_df['decision_binary'] = merged_df['decision'].apply(get_acceptance)

In [71]:
merged_df.groupby('rating_str').count().iloc[:, :2]

,title,decision
rating_str,,
,10552,10552
"10: Top 5% of accepted papers, seminal paper",13,13
1: Reject,722,722
1: Trivial or wrong,15,15
2: Strong rejection,110,110
3: Clear rejection,538,538
3: Weak Reject,2038,2038
4: Ok but not good enough - rejection,1326,1326
5: Marginally below acceptance threshold,1459,1459


In [67]:
merged_df.to_csv(workdir + 'all_preprocessed_data.csv', index=False)

In [42]:
complete_df

,pid,rid,uid,confidence_str,rating_str,review,paper_len,conclusion,conclusion_len,url,emails,source,authors,title
0,ICLR_2017_335,1,ICLR_2017_335_r1,3: The reviewer is fairly confident that the e...,"7: Good paper, accept",The paper is about channel sparsity in Convolu...,4222,We have proposed a new compression technique t...,1548,http://openreview.net/pdf/6514d69f5b4a80e843ab...,,,"Soravit Changpinyo, Mark Sandler, Andrey Zhmog...",The Power of Sparsity in Convolutional Neural ...
1,ICLR_2017_335,2,ICLR_2017_335_r2,4: The reviewer is confident but not absolutel...,5: Marginally below acceptance threshold,This paper aims to improve efficiency of convo...,4222,We have proposed a new compression technique t...,1548,http://openreview.net/pdf/6514d69f5b4a80e843ab...,,,"Soravit Changpinyo, Mark Sandler, Andrey Zhmog...",The Power of Sparsity in Convolutional Neural ...
2,ICLR_2017_335,3,ICLR_2017_335_r3,4: The reviewer is confident but not absolutel...,4: Ok but not good enough - rejection,The paper experiments with channel to channel ...,4222,We have proposed a new compression technique t...,1548,http://openreview.net/pdf/6514d69f5b4a80e843ab...,,,"Soravit Changpinyo, Mark Sandler, Andrey Zhmog...",The Power of Sparsity in Convolutional Neural ...
3,ICLR_2017_257,1,ICLR_2017_257_r1,4: The reviewer is confident but not absolutel...,"7: Good paper, accept",This paper investigates the impact of orthogon...,5152,We have explored a number of methods for contr...,773,http://openreview.net/pdf/6a2315bc79e028ba384c...,,,"Eugene Vorontsov, Chiheb Trabelsi, Samuel Kado...",On orthogonality and learning recurrent networ...
4,ICLR_2017_257,2,ICLR_2017_257_r2,4: The reviewer is confident but not absolutel...,5: Marginally below acceptance threshold,"The paper is well-motivated, and is part of a ...",5152,We have explored a number of methods for contr...,773,http://openreview.net/pdf/6a2315bc79e028ba384c...,,,"Eugene Vorontsov, Chiheb Trabelsi, Samuel Kado...",On orthogonality and learning recurrent networ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28042,NIPS_2019_955,2,NIPS_2019_955_r2,,,1. Originality. The authors propose a novel me...,4935,We propose the notion of performance gap to me...,1219,http://papers.nips.cc/paper/9249-transfer-lear...,,,"Boyu Wang, Jorge Mendez, Mingbo Cai, Eric Eaton",Transfer Learning via Minimizing the Performan...
28043,NIPS_2019_955,3,NIPS_2019_955_r3,,,I would consider Theorem 1 and analysis behind...,4935,We propose the notion of performance gap to me...,1219,http://papers.nips.cc/paper/9249-transfer-lear...,,,"Boyu Wang, Jorge Mendez, Mingbo Cai, Eric Eaton",Transfer Learning via Minimizing the Performan...
28044,NIPS_2019_1004,1,NIPS_2019_1004_r1,,,Originality - it's the first time that semi-su...,4777,We show that unlabeled data closes a sample co...,898,http://papers.nips.cc/paper/9298-unlabeled-dat...,,,"Yair Carmon, Aditi Raghunathan, Ludwig Schmidt...",Unlabeled Data Improves Adversarial Robustness
28045,NIPS_2019_1004,2,NIPS_2019_1004_r2,,,Originality: Previous theoretical work on the ...,4777,We show that unlabeled data closes a sample co...,898,http://papers.nips.cc/paper/9298-unlabeled-dat...,,,"Yair Carmon, Aditi Raghunathan, Ludwig Schmidt...",Unlabeled Data Improves Adversarial Robustness


In [43]:
all_df = pd.read_json(workdir + 'expanded_all_data.json')

In [44]:
all_df.shape

(22923, 14)

In [45]:
all_df.tail(2)

,title,decision,abstract,intro,conference,review,decision_label,intro_len,abstract_len,review_len,ce_extract,ce_extract_len,hybrid,hybrid_len
22921,Semi-Supervised Few-Shot Learning with Prototy...,Reject,Learning from a few examples is a key characte...,Humans are capable of learning complex skills ...,ICLR,This paper develops a random walk-based method...,0,626,272,880,Humans are capable of learning complex skills ...,991,Learning from a few examples is a key characte...,1263
22922,Semi-Supervised Few-Shot Learning with Prototy...,Reject,Learning from a few examples is a key characte...,Humans are capable of learning complex skills ...,ICLR,This submission introduces a version of the ra...,0,626,272,170,Humans are capable of learning complex skills ...,991,Learning from a few examples is a key characte...,1263


In [7]:
all_dfs

,title,decision,abstract,intro,conference,review,decision_label,intro_len,abstract_len,review_len,ce_extract,ce_extract_len,hybrid,hybrid_len
0,Active Learning from Imperfect Labelers,Accept,We study active learning where the labeler can...,"In active learning, the learner is given an in...",NIPS,This article presents a method for adaptive le...,1,813,99,189,Previous theoretical work on active learning h...,980,We study active learning where the labeler can...,1079
1,Active Learning from Imperfect Labelers,Accept,We study active learning where the labeler can...,"In active learning, the learner is given an in...",NIPS,This paper studies the problem of active learn...,1,813,99,199,Previous theoretical work on active learning h...,980,We study active learning where the labeler can...,1079
2,Active Learning from Imperfect Labelers,Accept,We study active learning where the labeler can...,"In active learning, the learner is given an in...",NIPS,The paper studies active learning where the la...,1,813,99,424,Previous theoretical work on active learning h...,980,We study active learning where the labeler can...,1079
3,Active Learning from Imperfect Labelers,Accept,We study active learning where the labeler can...,"In active learning, the learner is given an in...",NIPS,This paper focuses on active learning when the...,1,813,99,247,Previous theoretical work on active learning h...,980,We study active learning where the labeler can...,1079
4,Active Learning from Imperfect Labelers,Accept,We study active learning where the labeler can...,"In active learning, the learner is given an in...",NIPS,This paper provides some theoretical analysis ...,1,813,99,153,Previous theoretical work on active learning h...,980,We study active learning where the labeler can...,1079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22918,Model-Augmented Actor-Critic: Backpropagating ...,Accept (Poster),Current model-based reinforcement learning app...,Model-based reinforcement learning (RL) offers...,ICLR,After seeing the clarifications made to addres...,1,478,147,286,The horizon at which we apply the terminal Q-f...,846,Current model-based reinforcement learning app...,993
22919,Model-Augmented Actor-Critic: Backpropagating ...,Accept (Poster),Current model-based reinforcement learning app...,Model-based reinforcement learning (RL) offers...,ICLR,"This paper presents MAAC, a model-based reinfo...",1,478,147,347,The horizon at which we apply the terminal Q-f...,846,Current model-based reinforcement learning app...,993
22920,Semi-Supervised Few-Shot Learning with Prototy...,Reject,Learning from a few examples is a key characte...,Humans are capable of learning complex skills ...,ICLR,This paper suggests a method for semi-supervis...,0,626,272,223,Humans are capable of learning complex skills ...,991,Learning from a few examples is a key characte...,1263
22921,Semi-Supervised Few-Shot Learning with Prototy...,Reject,Learning from a few examples is a key characte...,Humans are capable of learning complex skills ...,ICLR,This paper develops a random walk-based method...,0,626,272,880,Humans are capable of learning complex skills ...,991,Learning from a few examples is a key characte...,1263


In [ ]:
for i, row in all_dfs.iterrows():
    

## Create Sample Text File for 

In [7]:
all_data = pd.read_csv(workdir + 'all_preprocessed_data.csv')

In [24]:
all_data = all_data[~all_data['review'].isna()]

In [25]:
all_data.shape

(22921, 29)

In [28]:
output_file_path = '/Users/erichansen/Dropbox/Mac/Desktop/Classes/CS282/ReviewAdvisor/tagger/all_reviews.txt'
with open(output_file_path, 'w') as outfile:
    for i, row in all_data.iterrows():
#         print(row['review'])
        try:
            review = row['review']
            outfile.write(review.replace('\n', '') + '\n')
        except Exception as e:
            print(i)
            print(row['review'])

In [9]:
# .to_csv()